# Playground

## GNN
This code receives multiple timeseries, transforms them into graphs, and then applies a GNN to them.
The graph embeddings are then used for downstream tasks.

In [1]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetModel, RandomDataModule, OtkaDataModule
from pytorch_lightning.loggers import TensorBoardLogger

## LSTM Autoencoder

In [ ]:
# Model

class Autoencoder(pl.LightningModule):
    def __init__(self, n_features, hidden_size):
        super().__init__()

        self.hidden_size = hidden_size

        self.encoder = nn.LSTM(n_features, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, n_features, batch_first=True)
        self.fc_decoder = nn.Linear(n_features, n_features)
        
        # b2b head
        self.b2b_decoder = nn.LSTM(hidden_size, n_features, batch_first=True)
        self.fc_b2b_decoder = nn.Linear(n_features, n_features)
        
        # classifier head
        self.cls = nn.Linear(hidden_size, 2)

    def forward(self, x):
        batch_size = x.size(0)
        n_timesteps = x.size(1)
        
        # autoencoder
        y_enc, (h_enc, c_enc) = self.encoder(x)
        x_enc = torch.rand(batch_size, n_timesteps, self.hidden_size)
        y_dec, (h_dec, c_dec) = self.decoder(x_enc, (h_enc, c_enc))
        y_dec = self.fc_decoder(y_dec)
        
        # b2b head
        x_enc_b2b = torch.rand(batch_size, n_timesteps, self.hidden_size)
        y_b2b, (h_b2b, c_b2b) = self.b2b_decoder(x_enc_b2b, (h_enc, c_enc))
        y_b2b = self.fc_b2b_decoder(y_b2b)
        
        # classifier head
        y_cls = self.cls(h_enc[-1, :, :])  # last hidden state of encoder
        
        return y_dec, y_b2b, y_cls

    def training_step(self, batch, batch_idx):
        X, subject_ids, y_b2b, y_cls = batch
        X_recon, y_b2b_hat, y_cls_hat = self(X)
        # loss
        loss_reconn = nn.functional.mse_loss(X_recon, X)
        loss_b2b = nn.functional.mse_loss(y_b2b_hat, y_b2b)
        loss_cls = nn.functional.cross_entropy(y_cls_hat, y_cls)
        loss = loss_reconn + loss_cls + loss_b2b
        #logging
        self.log('train/loss_reconn', loss_reconn)
        self.log('train/loss_b2b', loss_b2b)
        self.log('train/loss_cls', loss_cls)
        self.log('train/accuracy', (y_cls_hat.argmax(dim=1) == y_cls).float().mean())
        self.log('train/loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, subject_ids, y_b2b, y_cls = batch
        X_recon, y_b2b_hat, y_cls_hat = self(X)
        # loss
        loss_reconn = nn.functional.mse_loss(X_recon, X)
        loss_b2b = nn.functional.mse_loss(y_b2b_hat, y_b2b)
        loss_cls = nn.functional.cross_entropy(y_cls_hat, y_cls)
        loss = loss_reconn + loss_cls + loss_b2b
        #logging
        self.log('val/loss_reconn', loss_reconn)
        self.log('val/loss_b2b', loss_b2b)
        self.log('val/loss_cls', loss_cls)
        self.log('val/accuracy', (y_cls_hat.argmax(dim=1) == y_cls).float().mean())
        self.log('val/loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-2)

In [ ]:
# Experiment

segment_size = 120 * 3  # 3sec
batch_size = 256
n_features = 59
hidden_size = 59
max_epochs = 1000

datamodule = OtkaDataModule(segment_size=segment_size, batch_size=batch_size)

model = Autoencoder(n_features=n_features, hidden_size=hidden_size)

trainer = pl.Trainer(max_epochs=max_epochs,accelerator='cpu', log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)